In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none
pd.set_option('display.max_columns', None)

### I will create the interaction matrix from history data and save it in order to quick access

In [2]:
size = 'demo'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/train/history.parquet'

history_train_df = pd.read_parquet(history_train_path)
print(history_train_df.shape)

behaviors_train_path = f'./files/parquet/ebnerd_{size}/validation/behaviors.parquet'
behaviors_train_df = pd.read_parquet(behaviors_train_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_train_df.shape)


(1590, 5)
Raw demo validation behaviors df shape:           (25356, 17)


In [3]:
print(history_train_df.explode('impression_time_fixed').impression_time_fixed.min())
print(history_train_df.explode('impression_time_fixed').impression_time_fixed.max())
history_train_df.head(2)

2023-04-27 07:00:05
2023-05-18 06:59:51


,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."


In [4]:
print(behaviors_train_df.impression_time.min())
print(behaviors_train_df.impression_time.max())
behaviors_train_df.head(2)

2023-05-25 07:00:15
2023-06-01 06:59:33


,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,144772,NaN,2023-05-30 14:21:34,29.0,NaN,2,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],76658,False,NaN,NaN,NaN,False,29,7.0,59.0
1,144777,NaN,2023-05-30 14:22:11,10.0,NaN,2,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],76658,False,NaN,NaN,NaN,False,29,58.0,98.0


In [5]:
history_train_df.drop('impression_time_fixed', axis=1, inplace=True) # i will drop this column initialy even though i can proccess it and use it

In [6]:
history_train_df['scroll_percentage_fixed_len'] = history_train_df['scroll_percentage_fixed'].apply(lambda lst: len(lst))
history_train_df['article_id_fixed_len'] = history_train_df['article_id_fixed'].apply(lambda lst: len(lst))
history_train_df['read_time_fixed_len'] = history_train_df['read_time_fixed'].apply(lambda lst: len(lst))

In [7]:
history_train_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len
0,13538,"[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",582,582,582
1,58608,"[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,...",151,151,151


### The lenths of the lists in each column seem equal

In [7]:
history_train_df['scroll_percentage_fixed_has_0_in'] = history_train_df['scroll_percentage_fixed'].apply(lambda lst: 0 in lst)
history_train_df['read_time_fixed_has_0_in'] = history_train_df['read_time_fixed'].apply(lambda lst: 0 in lst)
history_train_df['read_time_fixed_has_nan'] = history_train_df['read_time_fixed'].apply(lambda lst: pd.isna(lst.any()))

In [8]:
history_train_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len,scroll_percentage_fixed_has_0_in,read_time_fixed_has_0_in,read_time_fixed_has_nan
0,13538,"[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",582,582,582,True,True,False
1,58608,"[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,...",151,151,151,False,True,False


In [9]:
history_train_df['read_time_fixed_has_nan'].unique()

array([False])

### There both nulls and zeros in the scroll_percentage_fixed column but not in read_time_fixed, i will use the read_time_fixed for interraction matrix or i will have to fix the nans to scroll_percentage_fixed

In [10]:
# # Combine interactions into a single metric (Potential use of scroll_percentage_fixed column)
# def combine_interactions(interactions, interactions2, w1=0.5, w2=0.5):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# history_train_df['interaction_combined'] = history_train_df.apply(lambda row:   combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed']) , axis=1)

# history_train_df.drop( ['scroll_percentage_fixed', 'read_time_fixed'], axis=1, inplace=True)

In [11]:
history_train_df.drop(['scroll_percentage_fixed_len', 'article_id_fixed_len', 'read_time_fixed_len', 'scroll_percentage_fixed_has_0_in', 'read_time_fixed_has_nan', 'scroll_percentage_fixed'], axis=1, inplace=True)

In [12]:
history_train_df.head(2)

,user_id,article_id_fixed,read_time_fixed,read_time_fixed_has_0_in
0,13538,"[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",True
1,58608,"[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,...",True


In [13]:
history_train_df = history_train_df.set_index('user_id').copy()

In [14]:
history_train_df.head(2)

,article_id_fixed,read_time_fixed,read_time_fixed_has_0_in
user_id,,,
13538,"[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",True
58608,"[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,...",True


In [15]:
history_train_df = history_train_df.explode('article_id_fixed')

In [16]:
history_train_df[history_train_df.index == 13538].shape

(582, 3)

In [17]:
history_train_df[history_train_df.index == 13538].head(2)

,article_id_fixed,read_time_fixed,read_time_fixed_has_0_in
user_id,,,
13538,9738663,"[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",True
13538,9738569,"[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",True


In [18]:
 # I rename the column in case i use two columns eventually and combine them. Also to keep the below code same
history_train_df.rename(columns= {'read_time_fixed': 'interaction_combined'}, inplace=True)

In [19]:
history_train_df.head(2)

,article_id_fixed,interaction_combined,read_time_fixed_has_0_in
user_id,,,
13538,9738663,"[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",True
13538,9738569,"[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",True


In [20]:
# Creation of interaction matrix

interaction_matrix = pd.DataFrame(columns=history_train_df.columns)
counter = 0
for user_id in tqdm(np.unique(history_train_df.index.values).tolist()):
    
    temp_df = history_train_df[history_train_df.index == user_id].copy()

    temp_lst = []

    # Fix the lists that exist in each cell and keep the values that i care
    temp_counter = 0
    for lst in list(temp_df['interaction_combined']):
        temp_lst.append(lst[temp_counter])
        temp_counter += 1
    
    temp_df['interaction_combined_fixed'] = temp_lst

    # Drop first unneccesary column
    temp_df.drop('interaction_combined', axis=1, inplace= True)
    temp_pivoted_df = temp_df.pivot_table(index = temp_df.index, columns='article_id_fixed', values='interaction_combined_fixed', aggfunc='sum').copy()

    if counter == 0:
        interaction_matrix = temp_pivoted_df.copy()
        #print(interaction_matrix)
    else:
        interaction_matrix = pd.concat([interaction_matrix, temp_pivoted_df], axis=0, ignore_index=False)
    
    counter += 1

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_2084/2062237798.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id in tqdm(np.unique(history_train_df.index.values).tolist()):


  0%|          | 0/1590 [00:00<?, ?it/s]

In [21]:
interaction_matrix.shape

(1590, 4247)

In [22]:
fillna_value = '0'
interaction_matrix.fillna(0, inplace=True)

In [23]:
# Save the interaction matrix to pickle
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'

interaction_matrix.to_pickle(interaction_matrix_file_path)

In [24]:
# Read interaction matrix pickle file
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)

In [25]:
interaction_matrix.head(2)

article_id_fixed,9667501,9676308,9735753,9739065,9756546,9757508,9758025,9759671,9759717,9763634,9764579,9764765,9765067,9423230,9757972,9759345,9763284,9763854,9763923,9764325,9764622,9765175,9765274,9766242,9766264,9766599,9767220,9767242,9767342,9767483,9767507,9767751,9767868,9767948,9768387,9768515,9769380,9769414,9769575,9770541,9701366,9733713,9733845,9736689,9737243,9738490,9738528,9738557,9738569,9740427,9740473,9745698,9745912,9746068,9747404,9747437,9747789,9749025,9749196,9749278,9750772,9752860,9754798,9754882,9755010,9755822,9755849,9756295,9756362,9756399,9757717,9757876,9765871,9765989,9766007,9766022,9767368,9767417,9767551,9767646,9767675,9767741,3218932,4514528,9322996,9685790,9729839,9729988,9730301,9737266,9738303,9738844,9738949,9739035,9739179,9739399,9739837,9740324,9740610,9740747,9741032,9741958,9742172,9742255,9742268,9742423,9743755,9743768,9744477,9744480,9744481,9744501,9745709,9745745,9745793,9746105,9746466,9747039,9747267,9747369,9748183,9748242,9748470,9748509,9748519,9748576,9748767,9748807,9749743,9749760,9750078,9750687,9750749,9751107,9751385,9751531,9751532,9751705,9751706,9751764,9751866,9752375,9752638,9752656,9752846,9753168,9753442,9754042,9754265,9755176,9755224,9755298,9755364,9755911,9755966,9756028,9756047,9756110,9756168,9756695,9757338,9757599,9757952,9758252,9758538,9759398,9759927,9761704,9761914,9762387,9762435,9762439,9765244,9765334,9765336,9765410,9765438,9765481,9765699,9765846,9765955,9766042,9766108,9766428,9766627,9766722,9766805,9767071,9767233,9767474,9768328,9768377,9769459,9769572,9769641,9770533,9690920,9714644,9716537,9725132,9725978,9726836,9728166,9730564,9735579,9736809,9736862,9737199,9737393,9737420,9738801,9738855,9738868,9738974,9739005,9739200,9739282,9739333,9739411,9739471,9739499,9739597,9739634,9739864,9739870,9739888,9740156,9740182,9740383,9740551,9740557,9740652,9740907,9740942,9740952,9740985,9741372,9741523,9741772,9741788,9741802,9742188,9742225,9742226,9742360,9742440,9742459,9742586,9742627,9742635,9742681,9742693,9742770,9743574,9743692,9743733,9743748,9743753,9743792,9743795,9743818,9743846,9743863,9743934,9743935,9743957,9744016,9744055,9744152,9744193,9744213,9744216,9744260,9744733,9744753,9744880,9744884,9744922,9745111,9745130,9745140,9745221,9745367,9745402,9745419,9745450,9745484,9745706,9745733,9745883,9745892,9746258,9746342,9746345,9746364,9746395,9746868,9746984,9747329,9747803,9747962,9747985,9748041,9748248,9748574,9748678,9748792,9748801,9748918,9748947,9748977,9749034,9749036,9749182,9749184,9749634,9749637,9749668,9749756,9749759,9749905,9749938,9750002,9750039,9750389,9750705,9750726,9750789,9750829,9750904,9750948,9750962,9750971,9751143,9751220,9751284,9751290,9751517,9751524,9751564,9751633,9751646,9751688,9751786,9751895,9751897,9751932,9751975,9751982,9752038,9752063,9752071,9752299,9752312,9752366,9752431,9752463,9752731,9752741,9752824,9752984,9752998,9753473,9753503,9753526,9753545,9753584,9754025,9754081,9754087,9754220,9754288,9754365,9754484,9754571,9754598,9754925,9754962,9755025,9755269,9755361,9755430,9755509,9755511,9755897,9755964,9755980,9756081,9756190,9756745,9756899,9757045,9757183,9757191,9757218,9757426,9757676,9757686,9757714,9757716,9757878,9757891,9757936,9758057,9758328,9758374,9758734,9758894,9759025,9759045,9759063,9759219,9759612,9759885,9760067,9760221,9760288,9760334,9760386,9760521,9760672,9760741,9760747,9760944,9761087,9761127,9761288,9761419,9761561,9761626,9761784,9761787,9761924,9761963,9762125,9762145,9762377,9762441,9762520,9762530,9762809,9762952,9762953,9762986,9763120,9763233,9763398,9763401,9763489,9763579,9763702,9763857,9763869,9763977,9764070,9764199,9764361,9764641,9764759,9764822,9765551,9765582,9765641,9765883,9765925,9766015,9766146,9766161,9766178,9766225,9766261,9766279,9766307,9766382,9766419,9766452,9766620,9766635,9766752,9766808,9766912,9766919,9766944,9766949,9767027,9767274,9767336,9767399,9767557,9767604,9767722,9768002,9768062,9768178,9768260,9768722,9768793,9768882

## Other

In [ ]:
# # Combine interactions into a single metric
# def combine_interactions(interactions, interactions2, w1=1, w2=1):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# # Flatten the interaction data with combined interactions
# flattened_data = []

# for _, row in history_train_df.iterrows():
#     user_id = row['user_id']
#     article_ids = row['article_id_fixed']
#     combined_interactions = combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed'])
    
#     for article_id, interaction_combined in zip(article_ids, combined_interactions):
#         flattened_data.append({
#             'user_id': user_id, 
#             'article_id_fixed': article_id, 
#             'interaction_combined': interaction_combined
#         })

# # Convert to DataFrame
# interaction_df = pd.DataFrame(flattened_data)